In [ ]:
# interactive
%matplotlib notebook
# QT exterenal window
# %matplotlib qt
# non interactive
# %matplotlib inline

# This examples shows who to download files from the ONC server
import glob
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.module
import strawb.tools

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

In [ ]:
# module_name = 'TUMSTANDARDMODULE001'

# get all possible dev_codes
dev_codes = set([i['dev_code'] for i in strawb.module_onc_id.values()])
# subtract not deployed dev_codes
dev_codes = dev_codes.difference(['TEST', 'UNITTEST', 'TUMSTANDARDMODULE002', 'TUMSTANDARDMODULE003'])

module_list = []

## Download the files from the ONC server

In [ ]:
onc_downloader = strawb.ONCDownloader(showInfo=False)

for dev_i in dev_codes:
    filters = {'deviceCode': dev_i,
               'dateFrom': '2021-01-01T00:00:00.000Z',
               'dateTo': '2022-01-01T23:59:59.000Z',
               'dataProductCode': 'SMRD',  # download only 'MODULE' data
               'extension': 'hdf5'}

    # download in foreground, (use 'onc_downloader.start(...)' for background)
    onc_downloader.download_file(filters=filters, allPages=True)

# create a combined HDF5 file

In [ ]:
for dev_i in dev_codes:
    # get a list with all filenames that should be included in the virtual HDF5 file
    f_names = glob.glob(f'{strawb.config_parser.config_parser.Config.raw_data_dir}/{module_name}_*MODULE.hdf5')
    f_names.sort()  # f_names should be sorted

    # create the virtual HDF5 file, f_names should be sorted, otherwise the data isn't sorted
    strawb.VirtualHDF5(f'{module_name}_MODULE.h5',
                       f_names,
                       obj_dict_filter=strawb.DatasetsInGroupSameSize)

# Load a file
## can be a single or combined aka. virtual HDF5 file

In [ ]:
# get all possible dev_codes
dev_codes = set([i['dev_code'] for i in strawb.module_onc_id.values()])
# subtract not deployed dev_codes
dev_codes = dev_codes.difference(['TEST', 'UNITTEST', 'TUMSTANDARDMODULE002', 'TUMSTANDARDMODULE003'])

module_list = []

for dev_i in dev_codes:
    module_list.append(strawb.sensors.module.FileHandler(f'{dev_i}_MODULE.h5'))

In [ ]:
module_list

In [ ]:
for module in module_list:
    print([i for i in module.file if 'lucifer' in i])

In [ ]:
module.accel_time[:]

In [ ]:
plt.figure()

plt.plot(module.accel_time[:] > 0)

In [ ]:
fig, ax = plt.subplots(nrows=8, squeeze=False, figsize=(10,15), sharex=True)
ax = ax.flatten()

i = 0

# ACCEL
strawb.tools.plot_binned_mean(module.accel_time, module.accel_x[:]*9.81, ax=ax[i], label='accel x')
strawb.tools.plot_binned_mean(module.accel_time, module.accel_y[:]*9.81, ax=ax[i], label='accel y')
ax[i].set_ylabel('Accel [m/s**2]')
i += 1
strawb.tools.plot_binned_mean(module.accel_time, module.accel_z[:]*9.81, ax=ax[i], label='accel z')
ax[i].set_ylabel('Accel [m/s**2]')
i += 1

# Magnet
strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_x, ax=ax[i], label='magneto x')
strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_y, ax=ax[i], label='magneto y')
strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_z, ax=ax[i], label='magneto z')
ax[i].set_ylabel('Magneto [a.u.]')
i += 1

# TEMP.
strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp1, ax=ax[i], label='temp 1')
strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp2, ax=ax[i], label='temp 2')
ax[i].set_ylabel('Temp [C]')
i += 1
strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp3, ax=ax[i], label='temp 3')
ax[i].set_ylabel('Temp [C]')
i += 1

# PTH
strawb.tools.plot_binned_mean(module.pth_time, module.pth_temperature, ax=ax[i], label='temp pth')
ax[i].set_ylabel('Temp [C]')
i += 1

strawb.tools.plot_binned_mean(module.pth_time, module.pth_humidity, ax=ax[i], label='pth humidity')
ax[i].set_ylabel('humidity [per]')
i += 1

strawb.tools.plot_binned_mean(module.pth_time, module.pth_pressure, ax=ax[i], label='pth pressure')
ax[i].set_ylabel('pressure [mbar]')
i += 1

for ax_i in ax:
    ax_i.legend(loc=1)
    ax_i.grid()

# # set limits
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
fig.autofmt_xdate()
locator = mdates.AutoDateLocator()
#fmt = mdates.AutoDateFormatter(locator, usetex=True)
fmt = mdates.ConciseDateFormatter(locator)
ax[-1].xaxis.set_major_formatter(fmt)
plt.tight_layout()

In [ ]:
limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
limits

In [ ]:
# get all possible dev_codes
dev_codes = set([i['dev_code'] for i in strawb.module_onc_id.values()])
# subtract not deployed dev_codes
dev_codes = dev_codes.difference(['TEST', 'UNITTEST', 'TUMSTANDARDMODULE002', 'TUMSTANDARDMODULE003'])

module_list = []

for dev_i in dev_codes:
    # get a list with all filenames that should be included in the virtual HDF5 file
    f_names = glob.glob(f'{strawb.config_parser.config_parser.Config.raw_data_dir}/{dev_i}_*MODULE.hdf5')
    f_names.sort()  # f_names should be sorted

    # create the virtual HDF5 file, f_names should be sorted, otherwise the data isn't sorted
    virtual_hdf5_gen = strawb.VirtualHDF5(f'{dev_i}_MODULE.h5',
                                          f_names,
                                          obj_dict_filter=strawb.DatasetsInGroupSameSize)

    module_list.append(strawb.sensors.module.FileHandler(f'{dev_i}_MODULE.h5'))

In [ ]:
fig, ax = plt.subplots(nrows=6, squeeze=False, figsize=(10,15), sharex=True)
ax = ax.flatten()


for module in module_list:
    print(module.module)
    i = 0
    # TEMP.
    mask = module.temperatures_time[:] > datetime.datetime(year=2020,month=12,day=1).timestamp()
    plot_time = module.temperatures_time[:][mask]
    strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp1[:][mask],
                                  ax=ax[i], label=f'temp 1 {module.module}')
    ax[i].set_ylabel('Temp [C]')
    i += 1

    strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp2[:][mask],
                                  ax=ax[i], label=f'temp 2 {module.module}')
    ax[i].set_ylabel('Temp [C]')
    i += 1

    strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp3[:][mask],
                                  ax=ax[i], label=f'temp 3 {module.module}')
    ax[i].set_ylabel('Temp [C]')
    i += 1

    # PTH
    mask = module.pth_time[:] > datetime.datetime(year=2020,month=12,day=1).timestamp()
    plot_time = module.pth_time[:][mask]
    strawb.tools.plot_binned_mean(plot_time, module.pth_temperature[:][mask],
                                  ax=ax[i], label=f'temp pth {module.module}')
    ax[i].set_ylabel('Temp [C]')
    i += 1

    strawb.tools.plot_binned_mean(plot_time, module.pth_humidity[:][mask],
                                  ax=ax[i], label=f'pth humidity {module.module}')
    ax[i].set_ylabel('humidity [per]')
    i += 1

    strawb.tools.plot_binned_mean(plot_time, module.pth_pressure[:][mask],
                                  ax=ax[i], label=f'pth pressure {module.module}')
    ax[i].set_ylabel('pressure [mbar]')
    i += 1

for ax_i in ax:
    ax_i.legend(loc=0)
    ax_i.grid()

# set limits
limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
fig.autofmt_xdate()
locator = mdates.AutoDateLocator()
#fmt = mdates.AutoDateFormatter(locator, usetex=True)
fmt = mdates.ConciseDateFormatter(locator)
ax[-1].xaxis.set_major_formatter(fmt)
plt.tight_layout()

In [ ]:
import datetime
date_str = [f_names_i.split(module_name+'_')[-1].split('.')[0] for f_names_i in f_names]
date_datetime = [datetime.datetime.strptime(date_i, '%Y%m%dT%H%M%S') for date_i in date_str]
date_np = np.array(date_datetime, dtype=np.datetime64)
size = np.array([os.path.getsize(f_names_i) for f_names_i in f_names])

In [ ]:
ax_i.set_ylim(200,400)

In [ ]:
plt.figure()
plt.plot(date_np, size)
plt.grid()